In [1]:
!pip install jpstat
import jpstat
import pandas as pd

In [2]:
stat = jpstat.estat.get_stat(key='9172300186e42e3c1e0acd0ce205021a2f27f28b')
jpstat.options["estat.api_key"] = '9172300186e42e3c1e0acd0ce205021a2f27f28b'

In [3]:
stat

,@id,STAT_NAME,GOV_ORG
0,00020111,民間企業の勤務条件制度等調査,人事院
1,00020112,国家公務員死因調査,人事院
2,00020131,国家公務員災害補償統計,人事院
3,00020151,退職公務員生活状況調査,人事院
4,00020211,一般職の国家公務員の任用状況調査,人事院
...,...,...,...
276,00650401,家庭からの二酸化炭素排出量の推計に係る実態調査 試験調査,環境省
277,00650402,環境保健サーベイランス調査,環境省
278,00650405,食品廃棄物等の発生抑制及び再生利用の促進の取組に係る実態調査,環境省
279,00650408,家庭部門のCO2排出実態統計調査,環境省


In [8]:
data, note = jpstat.estat.get_data(statsDataId="0004003462")

..

In [9]:
type(data)

pandas.core.frame.DataFrame

In [10]:
note

,@char,EXPLAIN
0,***,調査していないため該当数値がないもの
1,-,定義上該当数値がないもの


In [11]:
data.head(5)

,@unit,表章項目,移動後の住所地（現住地）2020～,国籍,移動前の住所地（前住地）2020～,時間軸,Value
0,人,移動前の住所地別転入者数,北海道,移動者,総数(前住地),2022年,234214
1,人,移動前の住所地別転入者数,北海道,移動者,北海道,2022年,179799
2,人,移動前の住所地別転入者数,北海道,移動者,札幌市,2022年,81662
3,人,移動前の住所地別転入者数,北海道,移動者,中央区,2022年,11696
4,人,移動前の住所地別転入者数,北海道,移動者,北区,2022年,10606
